In [1]:
# !sudo yum upgrade -y
# !sudo yum install libsndfile1 -y



Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
amzn2extra-docker                                        | 3.0 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
libnvidia-container/x86_64/signature                     |  833 B     00:00     
libnvidia-container/x86_64/signature                     | 2.1 kB     00:00 !!! 
neuron                                                   | 2.9 kB     00:00     
nvidia-container-runtime/x86_64/signature                |  833 B     00:00     
nvidia-conta

In [1]:
!pip install librosa
!pip install sagemaker --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import json
import time
import boto3
import numpy as np
import sagemaker
import sagemaker.huggingface

#BUCKET="[BUCKET_NAME]" # please use your bucket name
ROLE = sagemaker.get_execution_role()
sess = sagemaker.Session()
BUCKET = sess.default_bucket()
sagemaker
print(f"sagemaker role arn: {ROLE}")
print(f"sagemaker bucket: {BUCKET}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::348052051973:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole
sagemaker bucket: sagemaker-us-east-1-348052051973
sagemaker session region: us-east-1


In [31]:
from sagemaker.huggingface import HuggingFaceModel

model_name = 'whisper-zhtw4'
model_data = 's3://sagemaker-us-east-1-348052051973/whisper-hi/huggingface-whisper-training-1672924743/output/huggingface-whisper-training-1672924743/output/model.tar.gz'

huggingface_model = HuggingFaceModel(
        entry_point = 'inference.py',
        source_dir='./scripts',
        name = model_name,
        transformers_version='4.17.0',
        pytorch_version='1.10.2',
        py_version='py38',
        model_data=model_data,
        role=ROLE,
    )



In [32]:
# first time deploy as endpoint
# predictor = huggingface_model.deploy(
#     initial_instance_count=1,
#     instance_type="ml.m5d.2xlarge", 
#     endpoint_name = model_name,
# )

# update endpoint
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

sess.create_model(
    model_name,
    ROLE,
    huggingface_model.prepare_container_def(
        instance_type='ml.m5d.2xlarge'
    )
)

predictor = Predictor('whisper-zhtw', serializer=JSONSerializer(), deserializer=JSONDeserializer())
predictor.update_endpoint(model_name=model_name, initial_instance_count=1, instance_type='ml.m5d.2xlarge')

---------!

In [50]:
# inference audio file that download from S3 bucket or inference local audio file 
import soundfile
import IPython.display as ipd
import numpy as np
import librosa    


# s3.download_file(BUCKET, 'huggingface-blog/sample_audio/xxxxxx.wav', 'downloaded.wav')
# file_name ='downloaded.wav'

# download a sample audio file by using below link
#!wget https://datasets-server.huggingface.co/assets/common_voice/--/zh-TW/train/4/audio/audio.mp3
    
file_name ='audio.mp3'

speech_array, sampling_rate = librosa.load(file_name, sr=16000)


ipd.Audio(data=np.asarray(speech_array), autoplay=False, rate=sampling_rate)

In [51]:
%%time
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

json_request_data = {"speech_array": speech_array.tolist(),
                     "sampling_rate": sampling_rate}
predictor = Predictor('whisper-zhtw', serializer=JSONSerializer(), deserializer=JSONDeserializer())
prediction = predictor.predict(json_request_data)
print(prediction)

['["\\u4e5d\\u5341\\u65e5\\u5e63"]', 'application/json']
CPU times: user 112 ms, sys: 8.92 ms, total: 121 ms
Wall time: 9.16 s


In [52]:
result = json.loads(prediction[0])
result

['九十日幣']

In [ ]:
!sudo df -kh

In [14]:
!sudo find /tmp -type f -delete


In [ ]:
!sudo yum autoremove -y


In [ ]:
!cat ~/.sagemaker/config.yaml 

In [26]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = Predictor('whisper-zhtw', serializer=JSONSerializer(), deserializer=JSONDeserializer())
predictor.update_endpoint(model_name=model_name, initial_instance_count=1, instance_type='ml.m5d.2xlarge')

----------!

In [23]:
predictor.endpoint_context()


EndpointContext(sagemaker_session=<sagemaker.session.Session object at 0x7f80da156f80>,context_name='whisper-zhtw-1673009894-aws-endpoint',context_arn='arn:aws:sagemaker:us-east-1:348052051973:context/whisper-zhtw-1673009894-aws-endpoint',source=ContextSource(source_uri='arn:aws:sagemaker:us-east-1:348052051973:endpoint/whisper-zhtw',source_type='ARN',source_id='Fri Jan 06 12:58:14 UTC 2023'),context_type='Endpoint',properties={'DomainArn': 'arn:aws:sagemaker:us-east-1:348052051973:domain/d-0yyciqfkociv', 'FailureReason': 'Failed to extract model data archive from URL "s3://sagemaker-us-east-1-348052051973/whisper-zhtw2/model.tar.gz". The model data archive is too large. Please reduce the size of the model data archive or move to an instance type with more memory.', 'Status': 'Updating', 'UserProfileArn': 'arn:aws:sagemaker:us-east-1:348052051973:user-profile/d-0yyciqfkociv/a2'},creation_time=datetime.datetime(2023, 1, 6, 12, 58, 14, tzinfo=tzlocal()),created_by={},last_modified_time=d